# FINAL EXAM OF ECON 5821


#### 1155198448 PAN, Qiying

#### 1155198430 MA, Xinlan 

#### 1155207188 Huang, Lisha 

In our final exam, we will go though it the following procedure
* data process
* AR(1)
* LASSO & RIDGE
* Random Forest
* Gredient Boosting

In [1]:
# installing the package we need in this program
if (!require("readxl")) install.packages("readxl")
library(readxl)
if (!require("glmnet")) install.packages("glmnet")
library(glmnet)
if (!require("caret")) install.packages("caret")
library(readxl)
if (!require("tidyverse")) install.packages("tidyverse")
library(readxl)
if (!require("doParallel")) install.packages("doParallel")
library(doParallel)
if (!require("foreach")) install.packages("foreach")
library(foreach)
if (!require("gbm")) install.packages("gbm")
library(gbm)
if (!require("forecast")) install.packages("forecast")
library(forecast)
if (!require("tseries")) install.packages("tseries")
library(tseries)
if (!require("ranger")) install.packages("ranger")
library(ranger)
if (!require("DALEX")) install.packages("DALEX")
library(DALEX)

Loading required package: readxl

Loading required package: glmnet

Loading required package: Matrix

Loaded glmnet 4.1-8

Loading required package: caret

Loading required package: ggplot2

Loading required package: lattice

Loading required package: tidyverse

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ lubridate 1.9.3     ✔ tibble    3.2.1
✔ purrr     1.0.2     ✔ tidyr     1.3.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ tidyr::expand() masks Matrix::expand()
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
✖ purrr::lift()   masks caret::lift()
✖ tidyr::pack()   masks Matrix::pack()
✖ tidyr::unpack() masks Matrix::unpack()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Loading required package: doParallel

Loading required package: foreach


Att

In [2]:
cl <- makeCluster(16)  # Create a parallel cluster with 16 cores
registerDoParallel(cl) 

we download xlsx file and save it in the same folder

In [3]:
# process the data
data_train <- read_excel("US_PCE_training.xlsx")
# delet the NA value 
data_train = na.omit(data_train)
# delet the row of 'month'
data_train = data_train[data_train$month != 'month',]
data_train = t(data_train)

#  rename the variable to consist with 'xlsx' file
colnames(data_train) <- unlist(data_train[1, ])
#delet the first row, it the variable name of column
data_train <- data_train[-1, ] 

#head(data_train) 

In [4]:
# process the data.   the same procedure of above cell
data_score <- read_excel("US_PCE_testing_fake.xlsx")
data_score = na.omit(data_score)
data_score = data_score[data_score$month != 'month',]
data_score = t(data_score)
colnames(data_score) <- unlist(data_score[1, ])
data_score <- data_score[-1, ] 

#head(data_score)  

In [5]:
#check if  NA exist.
na_coords_train <- which(is.na(data_train), arr.ind = TRUE)
na_coords_score <- which(is.na(data_score), arr.ind = TRUE)
print(na_coords_train)
print(na_coords_score)

     row col
     row col


In [6]:
# Convert the data to dataframe format
data_train = as.data.frame(data_train)
data_score = as.data.frame(data_score)
# Convert the data to numeric
data_train[] <- lapply(data_train, as.numeric)
data_score[] <- lapply(data_score, as.numeric)

In [7]:
# creat copys
data_train_copy = data_train
data_score_copy = data_score

For x, we applied the same logarithmic transformation as we did for y.
$$y_
t = (\ln(PCE_t) - \ln(PCE_{t-1})) \times 12
$$

$$
x_{jt} = (\ln(\text{raw}X_{j,t}) - \ln(\text{raw}X_{j,t-1})) \times 12
$$


In [8]:

data_train = data_train_copy
data_score = data_score_copy 

data_score = rbind(tail(data_train,1),data_score)

#processed to log form
for (i in seq(1:ncol(data_train))){
    data_train[,i] = (log(data_train[,i])-log(lag(data_train[,i])))*12
    data_score[,i] = (log(data_score[,i])-log(lag(data_score[,i])))*12
}

#delect useless PCE
data_train$inflation = data_train$'Personal consumption expenditures'
data_score$inflation = data_score$'Personal consumption expenditures'
data_train = data_train[, !names(data_train) %in% "Personal consumption expenditures"]
data_score = data_score[, !names(data_score) %in% "Personal consumption expenditures"]

# remove inflation to first column
data_train <- data_train[, c(ncol(data_train), 1:(ncol(data_train)-1))]
data_score <- data_score[, c(ncol(data_score), 1:(ncol(data_score)-1))]

#add yt and delect NA value of first row
data_train$yt = data_train$inflation
data_score$yt = data_score$inflation
data_score = data_score[-1,]
#head(data_train)
#head(data_score)
#tail(data_train)
#tail(data_score)

the following cells is to process the data that need for 1 month predition, 3 month predition and 12 month prediction<br>Sorry for not streamlining the code by defining functions, which resulted in a lot of repetition in the subsequent parts. 

In [9]:
#data preparation for 1 month prediction
data_train_1 = data_train
data_score_1 = rbind(tail(data_train_1,1),data_score)   

data_train_1$inflation = lead(data_train_1$inflation,1) 
data_score_1$inflation = lead(data_score_1$inflation,1) 

data_train_1 <- data_train_1[1:(nrow(data_train_1) - 1), ]
data_score_1 <- data_score_1[1:(nrow(data_score_1) - 1), ]
data_train_1 = data_train_1[-1,]
#head(data_train_1)
#head(data_score_1)
#tail(data_train_1)
#tail(data_score_1)


this excel simply shows the processed data for one month prediction
| Dependent Variable Y| Independent variable X1 |
|---------|---------|
| \$Y_{t+1}\$ | \$X_t\$ | 
| \$Y_{t+2}\$ | \$X_{t+1}\$ |  

where  \$Y_{t+1}\$ stand for inflation in period t+1.

In [10]:
# data preparation for 3 month prediction.    Duplicated
data_train_3 = data_train
data_score_3 = rbind(tail(data_train_3,3),data_score)

data_train_3$inflation = lead(data_train_3$inflation,3) 
data_score_3$inflation = lead(data_score_3$inflation,3) 

data_train_3 <- data_train_3[1:(nrow(data_train_3) - 3), ]
data_score_3 <- data_score_3[1:(nrow(data_score_3) - 3), ]
data_train_3 = data_train_3[-1,]
#head(data_train_3)
#head(data_score_3)
#tail(data_train_3)
#tail(data_score_3)

In [11]:
# data preparation for 12 month prediction.    Duplicated
data_train_12 = data_train
data_score_12 = rbind(tail(data_train_12,12),data_score)

data_train_12$inflation = lead(data_train_12$inflation,12) 
data_score_12$inflation = lead(data_score_12$inflation,12) 

data_train_12 <- data_train_12[1:(nrow(data_train_12) - 12), ]
data_score_12 <- data_score_12[1:(nrow(data_score_12) - 12), ]
data_train_12 = data_train_12[-1,]
head(data_train_12)
#head(data_score_12)
#tail(data_train_12)
#tail(data_score_12)

,inflation,New domestic autos,New foreign autos,New light trucks,Net transactions in used autos,Used auto margin,Employee reimbursement,Used light trucks,Tires,Accessories and parts,⋯,Nonprofit nursing homes services to households,Recreation services to households,Education services to households,Social services to households,Religious organizations' services to households.1,Foundations and grantmaking and giving services to households.1,Services of social advocacy establishments to households,Civic and social organizations' services to households,Professional advocacy services to households,yt
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2,0.012444100,0.00000000,0.00000000,-0.0007398958,0.102295180,0.18317335,0.077363056,0.091356508,-0.021099937,-0.025997403,⋯,0.09992051,0.08441397,-0.1466413,-0.319770329,-0.24037707,-0.26360020,-0.320229841,-0.320276387,0.065499367,0.011864352
3,0.006994139,0.04619215,0.04617857,0.0454189326,0.143224207,-0.09242664,0.072780303,0.135275102,-0.001825194,0.028197986,⋯,-0.01531834,0.10051914,-0.1733776,-0.247234613,-0.20320526,-0.11411978,-0.246930622,-0.247079443,0.075966792,0.007903056
4,0.043427733,-0.02307385,-0.02306707,-0.0243497593,0.065531303,0.27284594,0.004087194,0.087327453,0.000260759,0.026814671,⋯,0.04589647,0.12565560,-0.1422115,-0.278072526,-0.21609165,-0.36656390,-0.277205914,-0.277330984,0.084891713,0.023677987
5,0.012379111,0.04610342,0.04608989,0.0453394079,-0.001037389,0.08804828,0.005107035,-0.001861331,-0.024274862,0.003512238,⋯,0.04952388,0.07640717,-0.1737625,-0.228198697,-0.22855828,-0.32628437,-0.229242413,-0.228846326,0.002685164,0.006306247
6,0.005411848,0.02298545,0.02297873,0.0227892371,0.107925077,0.26259253,0.071271209,0.089019804,0.003396214,-0.029006753,⋯,0.05499812,0.08988806,-0.1460290,-0.268576826,-0.24058803,-0.04458508,-0.270740889,-0.270040122,0.106916845,0.034625245
7,0.021623019,0.00000000,0.00000000,0.0000000000,0.238206066,0.08776410,-0.008123890,0.221970060,0.044323787,-0.150104350,⋯,0.05851368,0.10306457,0.0172724,0.001028762,0.08647907,-0.02779877,-0.002182017,-0.002905686,0.100696185,0.025119392


the following cells firstlly seperate the data to 70% train data and 30% test data.  then we also seperate the Y and X.<br> 
train 1 stand for 70% train data, test stand for 30% test data. score stand for real data post on May 16 

In [12]:
# train data and test data preparation     1 month predition
set.seed(123)  
train_size_1 <- floor(0.7 * nrow(data_train_1))

# Because it's time-series data, we split the training and testing sets directly based on sequence.
train_data_1 <- data_train_1[1:train_size_1, ]
test_data_1 <- data_train_1[(train_size_1 + 1):nrow(data_train_1), ]


train_x_1 <- data.matrix(train_data_1[,-1])  
train_y_1 <- train_data_1[,1]

test_x_1 <- data.matrix(test_data_1[,-1])
test_y_1 <- test_data_1[,1]

score_x_1 <- data.matrix(data_score_1[,-1])
score_y_1 <- data_score_1[,1]

In [13]:
# train data and test data preparation     3 month predition
train_size_3 <- floor(0.7 * nrow(data_train_3))

# Because it's time-series data, we split the training and testing sets directly based on sequence.
train_data_3 <- data_train_3[1:train_size_3, ]
test_data_3 <- data_train_3[(train_size_3 + 1):nrow(data_train_3), ]


train_x_3 <- data.matrix(train_data_3[,-1])  
train_y_3 <- train_data_3[,1]

test_x_3 <- data.matrix(test_data_3[,-1])
test_y_3 <- test_data_3[,1]

score_x_3 <- data.matrix(data_score_3[,-1])
score_y_3 <- data_score_3[,1]

In [14]:
# train data and test data preparation     12 month predition
train_size_12 <- floor(0.7 * nrow(data_train_12))

# Because it's time-series data, we split the training and testing sets directly based on sequence.
train_data_12 <- data_train_12[1:train_size_12, ]
test_data_12 <- data_train_12[(train_size_12 + 1):nrow(data_train_12), ]


train_x_12 <- data.matrix(train_data_12[,-1])  
train_y_12 <- train_data_12[,1]

test_x_12 <- data.matrix(test_data_12[,-1])
test_y_12 <- test_data_12[,1]

score_x_12 <- data.matrix(data_score_12[,-1])
score_y_12 <- data_score_12[,1]

# AR(1)

the following code show AR(1) model of 1 month predition, 3 month predition and 12 month predition.<br>
we simply use OLS method to estimate this model.
$$Y_t = c + \phi_1 Y_{t-1} + \epsilon_t$$in order to compare with the other meshine learning methods, we only use 70% train data to estimate this model. then we use this "trained" model to predict the test data.


In [15]:
# 1 month prediction of AR(1)
train_y = train_y_1
test_y = test_y_1
score_y = score_y_1

# train the AR(1) model using 70% train data
ar_model = lm(train_y[-1] ~ lag(train_y)[-1])


# train mse
prediction = numeric(length(lag(train_y)[-1]))   #Initialize a list to store each predicted value yt
train_y_ar = lag(train_y)[-1]
for (i in 1:length(lag(train_y)[-1])) {
  prediction[i] = coef(ar_model)[1]+coef(ar_model)[2]*train_y_ar[i]
}

MSE = mean((prediction - train_y[-1])^2) 
print(paste("MSE of the 70% train data (one month prediction): ", MSE))

# test mse
prediction = numeric(length(test_y))   #Initialize a list to store each predicted value yt
test_y_ar = c(tail(train_y,1),test_y[-length(test_y)])
for (i in 1:length(test_y)) {
  prediction[i] = coef(ar_model)[1]+coef(ar_model)[2]*test_y_ar[i]
}

MSE = mean((prediction - test_y)^2) # MSE of test data
print(paste("MSE of the rest 30% data (one month prediction): ", MSE))


#score mse
prediction_score = numeric(length(score_y))   # MSE of score data
score_y_ar = c(tail(test_y,1),score_y[-length(score_y)])
for (i in 1:length(score_y)) {
  prediction_score[i] = coef(ar_model)[1]+coef(ar_model)[2]*score_y_ar[i]
}

MSE = mean((prediction_score - score_y)^2)
print(paste("MSE of the real test data (one month prediction): ", MSE))



[1] "MSE of the 70% train data (one month prediction):  0.000378794438041514"
[1] "MSE of the rest 30% data (one month prediction):  0.000577229495917976"
[1] "MSE of the real test data (one month prediction):  78.2171299183431"


In [16]:
# 3 month prediction of AR(1)
#NOTE:  AR(1) only use dependent value, thus we let  'train_y = train_y_1 '   and then implement "lag(train_y,3)" function to estimate 3 month predition AR(1)
train_y = train_y_1
test_y = test_y_1
score_y = score_y_1

# train the AR(1) model using 70% train data
ar_model = lm(train_y[-(1:3)] ~ lag(train_y,3)[-(1:3)])


# train mse
prediction = numeric(length(lag(train_y,3)[-(1:3)]))   #Initialize a list to store each predicted value yt
train_y_ar = lag(train_y,3)[-(1:3)]
for (i in 1:length(lag(train_y,3)[-(1:3)])) {
  prediction[i] = coef(ar_model)[1]+coef(ar_model)[2]*train_y_ar[i]
}

MSE = mean((prediction - train_y[-(1:3)])^2) 
print(paste("MSE of the 70% train data (three month prediction): ", MSE))


# test mse
prediction = numeric(length(test_y))
test_y_ar = c(tail(train_y,3),test_y[1:(length(test_y)-3)])
for (i in 1:length(test_y)) {
  prediction[i] = coef(ar_model)[1]+coef(ar_model)[2]*test_y_ar[i]
}

MSE = mean((prediction - test_y)^2)
print(paste("MSE of the rest 30% data (three month prediction): ", MSE))


#score mse
prediction_score = numeric(length(score_y))
score_y_ar = c(tail(test_y,3),score_y[1:(length(score_y)-3)])
for (i in 1:length(score_y)) {
  prediction_score[i] = coef(ar_model)[1]+coef(ar_model)[2]*score_y_ar[i]
}

MSE = mean((prediction_score - score_y)^2)
print(paste("MSE of the real test data (three month prediction): ", MSE))

[1] "MSE of the 70% train data (three month prediction):  0.000488677768046148"
[1] "MSE of the rest 30% data (three month prediction):  0.000957063522439213"
[1] "MSE of the real test data (three month prediction):  49.2549575012447"


In [17]:
# 12 month prediction of AR(1)
#NOTE:  AR(1) only use dependent value, thus we let  'train_y = train_y_1 '   and then implement "lag(train_y,12)" function to estimate 12 month predition AR(1)
train_y = train_y_1
test_y = test_y_1
score_y = score_y_1

# train the AR(1) model using 70% train data
ar_model = lm(train_y[-(1:12)] ~ lag(train_y,12)[-(1:12)])


# train mse
prediction = numeric(length(lag(train_y,12)[-(1:12)]))   #Initialize a list to store each predicted value yt
train_y_ar = lag(train_y,12)[-(1:12)]
for (i in 1:length(lag(train_y,12)[-(1:12)])) {
  prediction[i] = coef(ar_model)[1]+coef(ar_model)[2]*train_y_ar[i]
}

MSE = mean((prediction - train_y[-(1:12)])^2) 
print(paste("MSE of the 70% train data (12 month prediction): ", MSE))

# test mse
prediction = numeric(length(test_y))
test_y_ar = c(tail(train_y,12),test_y[1:(length(test_y)-12)])
for (i in 1:length(test_y)) {
  prediction[i] = coef(ar_model)[1]+coef(ar_model)[2]*test_y_ar[i]
}

MSE = mean((prediction - test_y)^2)
print(paste("MSE of the rest 30% data (12 month prediction): ", MSE))



#score mse
prediction_score = numeric(length(score_y))
score_y_ar = c(tail(test_y,12),score_y[1:(length(score_y)-12)])
for (i in 1:length(score_y)) {
  prediction_score[i] = coef(ar_model)[1]+coef(ar_model)[2]*score_y_ar[i]
}

MSE = mean((prediction_score - score_y)^2)
print(paste("MSE of the real test data (12 month prediction): ", MSE))

[1] "MSE of the 70% train data (12 month prediction):  0.000574751635870192"
[1] "MSE of the rest 30% data (12 month prediction):  0.00101029907637994"
[1] "MSE of the real test data (12 month prediction):  42.0167810877226"


## Lasso and Ridge

in this part, we use caret package to help conduct CV for lasso and ridge model. Still,we use 70% train data to train Lasso and Ridge regression model. <br>firstly, we apply CV method to choose best lambda, and use the best lambda to train lasso(Ridge) model. since the data is time series data, we use Sliding Window Cross-Validation<br>then use the trained model to predict test data and score data

besides, except for AR(1) model, the input of all of other mechine learning models are all \$X_t\$ and \$Y_t\$. the output is  \$Y_{t+1}\$ for one month predition$$

In [15]:
# ridge and lasso  model for 1 month prediction
train_x = train_x_1
train_y = train_y_1
test_x = test_x_1
test_y = test_y_1
score_x=score_x_1
score_y = score_y_1

train_data = train_data_1
test_data = test_data_1
data_scoreMSE = data_score_1

set.seed(1)
# CV to choose best lambda.     Ridge

# Sliding Window Cross-Validation
train_control <- trainControl(method = "timeslice",
                              initialWindow = floor(0.7*nrow(train_x)),
                              horizon = 12,
                              skip = 3,
                              fixedWindow = TRUE,
                              allowParallel = TRUE)

#choose best lambda, alpha = 0 is ridge regression
tuneGrid <- expand.grid(
  alpha = 0,
  lambda = 10^seq(2, -5, length=10)
) 


model <- train(train_x, train_y,
               method = "glmnet",
               tuneGrid = tuneGrid,
               trControl = train_control);



print(paste("best lambda:",model$bestTune$lambda ))
ridge.mod = glmnet(train_x, train_y, alpha=0, lambda=model$bestTune$lambda)
ridge_1= ridge.mod

# train mse
ridge.pred <- predict(ridge.mod, newx = train_x )
mse <- mean((ridge.pred - train_data[, 1])^2)
print(paste("MSE of the 70% train data (1 month prediction): ", mse)) 

# test mse
ridge.pred <- predict(ridge.mod, newx = test_x )
mse <- mean((ridge.pred - test_data[, 1])^2)
print(paste("MSE of the rest 30% data (1 month prediction): ", mse)) 

# score mse
ridge.pred <- predict(ridge.mod,  newx = score_x )
mse_score <- mean((ridge.pred - data_scoreMSE[, 1])^2)
print(paste("MSE of real test data (1 month prediction): ", mse_score)) 

Warning message in nominalTrainWorkflow(x = x, y = y, wts = weights, info = trainInfo, :
“There were missing values in resampled performance measures.”


[1] "best lambda: 0.0774263682681127"
[1] "MSE of the 70% train data (1 month prediction):  0.000256384570880351"
[1] "MSE of the rest 30% data (1 month prediction):  0.000569638397594488"
[1] "MSE of real test data (1 month prediction):  32.4776603805927"


In [19]:
# CV to choose best lambda.     Lasso

# Sliding Window Cross-Validation
train_control <- trainControl(method = "timeslice",
                              initialWindow = floor(0.7*nrow(train_x)),
                              horizon = 1,
                              skip = 1,
                              fixedWindow = TRUE,
                              allowParallel = TRUE)

#choose best lambda, alpha = 1 is lasso regression
tuneGrid <- expand.grid(
  alpha = 1,
  lambda = 10^seq(2, -5, length=50)
) 


model <- train(train_x, train_y,
               method = "glmnet",
               tuneGrid = tuneGrid,
               trControl = train_control);

print(paste("best lambda:",model$bestTune$lambda ))
lasso.mod = glmnet(train_x, train_y, alpha=1, lambda=model$bestTune$lambda)
lasso_1 = lasso.mod
# train mse
lasso.pred <- predict(lasso.mod, newx = train_x )
mse <- mean((lasso.pred - train_data[, 1])^2)
print(paste("MSE of the rest 70% train data (1 month prediction): ", mse)) 

# test mse
lasso.pred <- predict(lasso.mod, newx = test_x )
mse <- mean((lasso.pred - test_data[, 1])^2)
print(paste("MSE of the rest 30% data (1 month prediction): ", mse)) 

# score mse
lasso.pred <- predict(lasso.mod,  newx = score_x )
mse_score <- mean((lasso.pred - data_scoreMSE[, 1])^2)
print(paste("MSE of real test data (1 month prediction): ", mse_score)) 

Warning message in nominalTrainWorkflow(x = x, y = y, wts = weights, info = trainInfo, :
“There were missing values in resampled performance measures.”


[1] "best lambda: 0.00138949549437314"
[1] "MSE of the rest 70% train data (1 month prediction):  0.000262347557236409"
[1] "MSE of the rest 30% data (1 month prediction):  0.000549502795643353"
[1] "MSE of real test data (1 month prediction):  40.80857849421"


In [16]:
# ridge and lasso  model for 3 month prediction
train_x = train_x_3
train_y = train_y_3
test_x = test_x_3
test_y = test_y_3
score_x=score_x_3
score_y = score_y_3

train_data = train_data_3
test_data = test_data_3
data_scoreMSE = data_score_3


# CV to choose best lambda.     Ridge

# Sliding Window Cross-Validation
train_control <- trainControl(method = "timeslice",
                              initialWindow = floor(0.7*nrow(train_x)),
                              horizon = 1,
                              skip = 1,
                              fixedWindow = TRUE,
                              allowParallel = TRUE)

#choose best lambda
tuneGrid <- expand.grid(
  alpha = 0,   #ridge
  lambda = 10^seq(2, -5, length=50)
) 


model <- train(train_x, train_y,
               method = "glmnet",
               tuneGrid = tuneGrid,
               trControl = train_control);


print(paste("best lambda:",model$bestTune$lambda ))
ridge.mod = glmnet(train_x, train_y, alpha=0, lambda=model$bestTune$lambda)
ridge_3 = ridge.mod
# train mse
ridge.pred <- predict(ridge.mod, newx = train_x )
mse <- mean((ridge.pred - train_data[, 1])^2)
print(paste("MSE of the 70% train data (3 month prediction): ", mse)) 


# test mse
ridge.pred <- predict(ridge.mod, newx = test_x )
mse <- mean((ridge.pred - test_data[, 1])^2)
print(paste("MSE of the rest 30% data (3 month prediction): ", mse)) 


# score mse
ridge.pred <- predict(ridge.mod,  newx = score_x )
mse_score <- mean((ridge.pred - data_scoreMSE[, 1])^2)
print(paste("MSE of real test data (3 month prediction): ", mse_score)) 

Warning message in nominalTrainWorkflow(x = x, y = y, wts = weights, info = trainInfo, :
“There were missing values in resampled performance measures.”


[1] "best lambda: 0.1"
[1] "MSE of the 70% train data (3 month prediction):  0.00032589367791192"
[1] "MSE of the rest 30% data (3 month prediction):  0.000969901538729414"
[1] "MSE of real test data (3 month prediction):  32.4111243965841"


In [17]:
# CV to choose best lambda.     Lasso

# Sliding Window Cross-Validation
train_control <- trainControl(method = "timeslice",
                              initialWindow = floor(0.7*nrow(train_x)),
                              horizon = 1,
                              skip = 1,
                              fixedWindow = TRUE,
                              allowParallel = TRUE)

# choose best lambda
tuneGrid <- expand.grid(
  alpha = 1,  #lasso
  lambda = 10^seq(2, -5, length=100)
) 


model <- train(train_x, train_y,
               method = "glmnet",
               tuneGrid = tuneGrid,
               trControl = train_control);


print(paste("best lambda:",model$bestTune$lambda ))
lasso.mod = glmnet(train_x, train_y, alpha=1, lambda=model$bestTune$lambda)
lasso_3 = lasso.mod
# train mse
lasso.pred <- predict(lasso.mod, newx = train_x )
mse <- mean((lasso.pred - train_data[, 1])^2)
print(paste("MSE of the rest 70% train data (3 month prediction): ", mse)) 

# test mse
lasso.pred <- predict(lasso.mod, newx = test_x )
mse <- mean((lasso.pred - test_data[, 1])^2)
print(paste("MSE of the rest 30% data (3 month prediction): ", mse)) 

# test mse
lasso.pred <- predict(lasso.mod,  newx = score_x )
mse_score <- mean((lasso.pred - data_scoreMSE[, 1])^2)
print(paste("MSE of real test data (3 month prediction): ", mse_score)) 

Warning message in nominalTrainWorkflow(x = x, y = y, wts = weights, info = trainInfo, :
“There were missing values in resampled performance measures.”


[1] "best lambda: 0.00215443469003189"
[1] "MSE of the rest 70% train data (3 month prediction):  0.000353570228509921"
[1] "MSE of the rest 30% data (3 month prediction):  0.000731679051683768"
[1] "MSE of real test data (3 month prediction):  36.331982194513"


In [20]:
# ridge and lasso  model for 12 month prediction
train_x = train_x_12
train_y = train_y_12
test_x = test_x_12
test_y = test_y_12
score_x=score_x_12
score_y = score_y_12

train_data = train_data_12
test_data = test_data_12
data_scoreMSE = data_score_12


# CV to choose best lambda.    ridge

# Sliding Window Cross-Validation
train_control <- trainControl(method = "timeslice",
                              initialWindow = floor(0.7*nrow(train_x)),
                              horizon = 1,
                              skip = 1,
                              fixedWindow = TRUE,
                              allowParallel = TRUE)

#choose best lambda
tuneGrid <- expand.grid(
  alpha = 0,  #ridge
  lambda = 10^seq(2, -5, length=100)
) 


model <- train(train_x, train_y,
               method = "glmnet",
               tuneGrid = tuneGrid,
               trControl = train_control);


print(paste("best lambda:",model$bestTune$lambda ))
ridge.mod = glmnet(train_x, train_y, alpha=0, lambda=model$bestTune$lambda)
ridge_12 = ridge.mod
# train mse
ridge.pred <- predict(ridge.mod, newx = train_x )
mse <- mean((ridge.pred - train_data[, 1])^2)
print(paste("MSE of the 70% train data (12 month prediction): ", mse)) 


#test mse
ridge.pred <- predict(ridge.mod, newx = test_x )
mse <- mean((ridge.pred - test_data[, 1])^2)
print(paste("MSE of the rest 30% data (12 month prediction): ", mse)) 


#score mse
ridge.pred <- predict(ridge.mod,  newx = score_x )
mse_score <- mean((ridge.pred - data_scoreMSE[, 1])^2)
print(paste("MSE of real test data (12 month prediction): ", mse_score)) 

Warning message in nominalTrainWorkflow(x = x, y = y, wts = weights, info = trainInfo, :
“There were missing values in resampled performance measures.”


[1] "best lambda: 0.0403701725859656"
[1] "MSE of the 70% train data (12 month prediction):  0.000367960356244403"
[1] "MSE of the rest 30% data (12 month prediction):  0.000874906210777779"
[1] "MSE of real test data (12 month prediction):  32.0250915395778"


In [22]:
# CV to choose best lambda.    lasso

# Sliding Window Cross-Validation
train_control <- trainControl(method = "timeslice",
                              initialWindow = floor(0.7*nrow(train_x)),
                              horizon = 1,
                              skip = 1,
                              fixedWindow = TRUE,
                              allowParallel = TRUE)

#choose best lambda
tuneGrid <- expand.grid(
  alpha = 1, #lasso
  lambda = 10^seq(2, -5, length=100)
) 


model <- train(train_x, train_y,
               method = "glmnet",
               tuneGrid = tuneGrid,
               trControl = train_control);




print(paste("best lambda:",model$bestTune$lambda ))
lasso.mod = glmnet(train_x, train_y, alpha=1, lambda=model$bestTune$lambda)
lasso_12 = lasso.mod
# train mse
lasso.pred <- predict(lasso.mod, newx = train_x )
mse <- mean((lasso.pred - train_data[, 1])^2)
print(paste("MSE of the rest 70% train data (12 month prediction): ", mse)) 

#test mse
lasso.pred <- predict(lasso.mod, newx = test_x )
mse <- mean((lasso.pred - test_data[, 1])^2)
print(paste("MSE of the rest 30% data (12 month prediction): ", mse)) 

# score mse
lasso.pred <- predict(lasso.mod,  newx = score_x )
mse_score <- mean((lasso.pred - data_scoreMSE[, 1])^2)
print(paste("MSE of real test data (12 month prediction): ", mse_score)) 

Warning message in nominalTrainWorkflow(x = x, y = y, wts = weights, info = trainInfo, :
“There were missing values in resampled performance measures.”


[1] "best lambda: 0.00183073828029537"
[1] "MSE of the rest 70% train data (12 month prediction):  0.00041115815315606"
[1] "MSE of the rest 30% data (12 month prediction):  0.000702975101444752"
[1] "MSE of real test data (12 month prediction):  31.3842401713715"


## Random Forest

in this part, we use caret package to help conduct CV. Still,we use 70% train data to train RF model. <br>firstly, we apply CV method to choose best m and n.trees, and use the best tunes to train RF model. since the data is time series data, we use Sliding Window Cross-Validation<br>then use the trained model to predict test data and score data

In [24]:
# 1 month prediction for RF model

train_x = train_x_1
train_y = train_y_1
test_x = test_x_1
test_y = test_y_1
score_y = score_y_1

train_data = train_data_1
test_data = test_data_1
data_scoreMSE = data_score_1


# initialise a data.frame to store best tune M, and RMSE
rftune <- data.frame()
rftune$rmse <- c()
rftune$mtry <- c()

set.seed(1)


star = Sys.time()

# tuneGrid parameter only allowed 3 input "mtry" "splitrule" and 'min.node.size'
# thus, we write a loop to find another best tune  'n.trees'.
# we need to find best mtry and n.trees

for (i in seq(1,5)){   
train_control <- trainControl(method = "timeslice",
              initialWindow = floor(0.7*nrow(train_x)),
              horizon = 12,
              skip = 3,
              fixedWindow = TRUE,
              allowParallel = TRUE)


tuneGrid <- expand.grid(
                       mtry =floor(ncol(train_x)/seq(1,10)), # best mtry
                       splitrule= 'variance',
                       min.node.size = 5) 

model <- train(train_x, train_y,
   method = "ranger",
   tuneGrid = tuneGrid,
   trControl = train_control,
   num.trees = 100*i  # best n.trees
              );


# save the best RMSE and mtry in every loop
rftune[i,'rmse'] = min(model$results$RMSE)
rftune[i,'mtry'] = model$bestTune$mtry
}

end = Sys.time()

# cv time
print(paste("total cv time", end-star))

[1] "total cv time 1.36939302285512"


In [25]:
best_rfmodel <- ranger(x= train_x, y = train_y,  
                     data = train_data,  
                     num.trees = 100*which.min(rftune$rmse),   #best tree number
                     mtry = rftune[which.min(rftune$rmse),'mtry'],  #best mtry 
                     min.node.size = 5)
rf_1= best_rfmodel
print(paste(c('tree number:','mtry:'),c(100*which.min(rftune$rmse),rftune[which.min(rftune$rmse),'mtry']) ))
#train mse
predictions <- predict(best_rfmodel, data = train_data)
mse <- mean((predictions$predictions - train_data[, 1])^2)
print(paste("MSE of the 70% train data (1 month prediction): ", mse))


#test mse
predictions <- predict(best_rfmodel, data = test_data)
mse <- mean((predictions$predictions - test_data[, 1])^2)
print(paste("MSE of the rest 30% data (1 month prediction): ", mse))

#score mse
predictions <- predict(best_rfmodel, data = data_scoreMSE)
mse <- mean((predictions$predictions - data_scoreMSE[, 1])^2)
print(paste("MSE of real test data (1 month prediction):  ", mse))



[1] "tree number: 500" "mtry: 22"        
[1] "MSE of the 70% train data (1 month prediction):  4.75745230389106e-05"
[1] "MSE of the rest 30% data (1 month prediction):  0.000604261684359342"
[1] "MSE of real test data (1 month prediction):   32.1727726377268"


In [26]:
# 3 month prediction for RF model

train_x = train_x_3
train_y = train_y_3
test_x = test_x_3
test_y = test_y_3
score_y = score_y_3

train_data = train_data_3
test_data = test_data_3
data_scoreMSE = data_score_3


# initialise a data.frame to store best tune M, and RMSE
rftune <- data.frame()
rftune$rmse <- c()
rftune$mtry <- c()

set.seed(1)

star = Sys.time()

# tuneGrid parameter only allowed 3 input "mtry" "splitrule" and 'min.node.size'
# thus, we write a loop to find another best tune  'n.trees'.
# we need to find best mtry and n.trees
for (i in seq(1,5)){
train_control <- trainControl(method = "timeslice",
              initialWindow = floor(0.7*nrow(train_x)),
              horizon = 12,
              skip = 3,
              fixedWindow = TRUE,
              allowParallel = TRUE)


tuneGrid <- expand.grid(
                       mtry =floor(ncol(train_x)/seq(1,10)),
                       splitrule= 'variance',
                       min.node.size = 5) 

model <- train(train_x, train_y,
   method = "ranger",
   tuneGrid = tuneGrid,
   trControl = train_control,
   num.trees = 100*i);



rftune[i,'rmse'] = min(model$results$RMSE)
rftune[i,'mtry'] = model$bestTune$mtry
}

end = Sys.time()

#cv time
print(paste("total cv time", end-star))

[1] "total cv time 1.35141151746114"


In [27]:
best_rfmodel <- ranger(x= train_x, y = train_y,   
                     data = train_data,  
                     num.trees = 100*which.min(rftune$rmse), #best tree number
                     mtry = rftune[which.min(rftune$rmse),'mtry'], #best mtry
                     min.node.size = 5)
rf_3= best_rfmodel
print(paste(c('tree number:','mtry:'),c(100*which.min(rftune$rmse),rftune[which.min(rftune$rmse),'mtry']) ))
#train mse
predictions <- predict(best_rfmodel, data = train_data)
mse <- mean((predictions$predictions - train_data[, 1])^2)
print(paste("MSE of the 70% train data (3 month prediction): ", mse))

#test mse
predictions <- predict(best_rfmodel, data = test_data)
mse <- mean((predictions$predictions - test_data[, 1])^2)
print(paste("MSE of the rest 30% data (3 month prediction): ", mse))

#score mse 
predictions <- predict(best_rfmodel, data = data_scoreMSE)
mse <- mean((predictions$predictions - data_scoreMSE[, 1])^2)
print(paste("MSE of real test data (3 month prediction):  ", mse))

[1] "tree number: 300" "mtry: 22"        
[1] "MSE of the 70% train data (3 month prediction):  5.47087060649134e-05"
[1] "MSE of the rest 30% data (3 month prediction):  0.000705183946571147"
[1] "MSE of real test data (3 month prediction):   32.1487636037803"


In [28]:
# 12 month prediction for RF model

train_x = train_x_12
train_y = train_y_12
test_x = test_x_12
test_y = test_y_12
score_y = score_y_12

train_data = train_data_12
test_data = test_data_12
data_scoreMSE = data_score_12


# initialise a data.frame to store best tune M, and RMSE
rftune <- data.frame()
rftune$rmse <- c()
rftune$mtry <- c()

set.seed(1)

star = Sys.time()

# tuneGrid parameter only allowed 3 input: "mtry" "splitrule" and 'min.node.size'
# thus, we write a loop to find another best tune  'n.trees'.
# we need to find best mtry and n.trees
for (i in seq(1,5)){
train_control <- trainControl(method = "timeslice",
              initialWindow = floor(0.7*nrow(train_x)),
              horizon = 12,
              skip = 3,
              fixedWindow = TRUE,
              allowParallel = TRUE)


tuneGrid <- expand.grid(
                       mtry =floor(ncol(train_x)/seq(1,10)),
                       splitrule= 'variance',
                       min.node.size = 5) 

model <- train(train_x, train_y,
   method = "ranger",
   tuneGrid = tuneGrid,
   trControl = train_control,
   num.trees = 100*i);



rftune[i,'rmse'] = min(model$results$RMSE)
rftune[i,'mtry'] = model$bestTune$mtry
}

end = Sys.time()

print(paste("total cv time", end-star))

[1] "total cv time 1.3377712726593"


In [29]:

best_rfmodel <- ranger(x= train_x, y = train_y,   
                     data = train_data,  
                     num.trees = 100*which.min(rftune$rmse),  # best tree number
                     mtry = rftune[which.min(rftune$rmse),'mtry'], #best mtry
                     min.node.size = 5)

rf_12 = best_rfmodel
print(paste(c('tree number:','mtry:'),c(100*which.min(rftune$rmse),rftune[which.min(rftune$rmse),'mtry']) ))
# train mse
predictions <- predict(best_rfmodel, data = train_data)
mse <- mean((predictions$predictions - train_data[, 1])^2)
print(paste("MSE of the 70% train data (12 month prediction): ", mse))

# test mse
predictions <- predict(best_rfmodel, data = test_data)
mse <- mean((predictions$predictions - test_data[, 1])^2)
print(paste("MSE of the rest 30% data (12 month prediction): ", mse))

# score mse
predictions <- predict(best_rfmodel, data = data_scoreMSE)
mse <- mean((predictions$predictions - data_scoreMSE[, 1])^2)
print(paste("MSE of real test data (12 month prediction):  ", mse))

[1] "tree number: 500" "mtry: 205"       
[1] "MSE of the 70% train data (12 month prediction):  6.3993172053716e-05"
[1] "MSE of the rest 30% data (12 month prediction):  0.000627703638032019"
[1] "MSE of real test data (12 month prediction):   32.111275902486"


## Gradient Boosting

in this part, we use caret package to help conduct CV. Still,we use 70% train data to train RF model. <br>firstly, we apply CV method to choose best depth, n.trees and shrinkage, and use the best tunes to train GBRT model. since the data is time series data, we use Sliding Window Cross-Validation<br>then use the trained model to predict test data and score data

In [30]:
# 1 month predition of gradient Boosting
train_x = train_x_1
train_y = train_y_1
test_x = test_x_1
test_y = test_y_1
score_y = score_y_1

train_data = train_data_1
test_data = test_data_1
data_scoreMSE = data_score_1

set.seed(1)


star = Sys.time()
train_control <- trainControl(method = "timeslice",
              initialWindow = floor(0.7*nrow(train_x)),
              horizon = 12,skip =3,
              fixedWindow = TRUE,
              allowParallel = TRUE)

# generate a tune grid 
gbmGrid <- expand.grid(
  interaction.depth = seq(1, 5),
  n.trees = c(50,100,200),
  shrinkage = c(0.01,0.1),
  n.minobsinnode = 10
)


model <- train(train_x, train_y,
   method = "gbm",
   tuneGrid=gbmGrid,
   trControl = train_control);

end = Sys.time()


print(paste("total cv time", end-star))
print(model$bestTune)


Warning message in nominalTrainWorkflow(x = x, y = y, wts = weights, info = trainInfo, :
“There were missing values in resampled performance measures.”


Iter   TrainDeviance   ValidDeviance   StepSize   Improve
     1        0.0008            -nan     0.1000    0.0001
     2        0.0007            -nan     0.1000    0.0001
     3        0.0006            -nan     0.1000    0.0001
     4        0.0005            -nan     0.1000    0.0001
     5        0.0005            -nan     0.1000    0.0000
     6        0.0005            -nan     0.1000    0.0000
     7        0.0004            -nan     0.1000    0.0000
     8        0.0004            -nan     0.1000    0.0000
     9        0.0004            -nan     0.1000    0.0000
    10        0.0003            -nan     0.1000    0.0000
    20        0.0002            -nan     0.1000    0.0000
    40        0.0001            -nan     0.1000   -0.0000
    50        0.0001            -nan     0.1000   -0.0000

[1] "total cv time 7.64680600166321"
   n.trees interaction.depth shrinkage n.minobsinnode
25      50                 4       0.1             10


In [31]:
gbm_model <- gbm(train_y ~ ., data = train_data, distribution = "gaussian", n.trees = model$bestTune$n.trees, interaction.depth = model$bestTune$interaction.depth,shrinkage = model$bestTune$shrinkage)

gb_1= gbm_model
print(paste(c('tree number:','depth:','shrinkage:'),c(model$bestTune$n.trees, model$bestTune$interaction.depth,model$bestTune$shrinkage)))
# train mse
predictions <- predict(gbm_model, newdata = train_data, type = "response")
mse <- mean((predictions - train_data[, 1])^2)
print(paste("MSE of the 70% train data (1 month prediction): ", mse))

# test mse
predictions <- predict(gbm_model, newdata = test_data, type = "response")
mse <- mean((predictions - test_data[, 1])^2)
print(paste("MSE of the rest 30% data (1 month prediction): ", mse))

# score mse
predictions <- predict(gbm_model, newdata = data_scoreMSE, type = "response")
mse_score <- mean((predictions - data_scoreMSE[, 1])^2)
print(paste("MSE of real test data (1 month prediction): ", mse_score))

[1] "tree number: 50" "depth: 4"        "shrinkage: 0.1" 


Using 50 trees...




[1] "MSE of the 70% train data (1 month prediction):  4.0238817465495e-06"


Using 50 trees...




[1] "MSE of the rest 30% data (1 month prediction):  0.000147180953317436"


Using 50 trees...




[1] "MSE of real test data (1 month prediction):  31.5720758745738"


In [32]:
# 3 month predition of gradient Boosting
train_x = train_x_3
train_y = train_y_3
test_x = test_x_3
test_y = test_y_3
score_y = score_y_3

train_data = train_data_3
test_data = test_data_3
data_scoreMSE = data_score_3

set.seed(3)

star = Sys.time()
train_control <- trainControl(method = "timeslice",
              initialWindow = floor(0.7*nrow(train_x)),
              horizon = 12,skip =3,
              fixedWindow = TRUE,
              allowParallel = TRUE)

#generate a tune grid
gbmGrid <- expand.grid(
  interaction.depth = seq(1, 5),
  n.trees = c(50,100,200),
  shrinkage = c(0.01,0.1),
  n.minobsinnode = 10
)


model <- train(train_x, train_y,
   method = "gbm",
   tuneGrid=gbmGrid,
   trControl = train_control);
end = Sys.time()

print(paste("total cv time", end-star))
print(model$bestTune)



Iter   TrainDeviance   ValidDeviance   StepSize   Improve
     1        0.0008            -nan     0.1000    0.0001
     2        0.0007            -nan     0.1000    0.0001
     3        0.0007            -nan     0.1000    0.0000
     4        0.0006            -nan     0.1000    0.0000
     5        0.0006            -nan     0.1000    0.0000
     6        0.0005            -nan     0.1000    0.0000
     7        0.0005            -nan     0.1000    0.0000
     8        0.0005            -nan     0.1000    0.0000
     9        0.0004            -nan     0.1000    0.0000
    10        0.0004            -nan     0.1000    0.0000
    20        0.0003            -nan     0.1000    0.0000
    40        0.0002            -nan     0.1000   -0.0000
    50        0.0002            -nan     0.1000   -0.0000

[1] "total cv time 7.57266211509705"
   n.trees interaction.depth shrinkage n.minobsinnode
22      50                 3       0.1             10


In [33]:
gbm_model <- gbm(train_y ~ ., data = train_data, distribution = "gaussian", n.trees = model$bestTune$n.trees, interaction.depth = model$bestTune$interaction.depth,shrinkage = model$bestTune$shrinkage)

gb_3 = gbm_model
print(paste(c('tree number:','depth:','shrinkage:'),c(model$bestTune$n.trees, model$bestTune$interaction.depth,model$bestTune$shrinkage)))
# train mse
predictions <- predict(gbm_model, newdata = train_data, type = "response")
mse <- mean((predictions - train_data[, 1])^2)
print(paste("MSE of the 70% train data (3 month prediction): ", mse))

# test mse
predictions <- predict(gbm_model, newdata = test_data, type = "response")
mse <- mean((predictions - test_data[, 1])^2)
print(paste("MSE of the rest 30% data (3 month prediction): ", mse))

# score mse
predictions <- predict(gbm_model, newdata = data_scoreMSE, type = "response")
mse_score <- mean((predictions - data_scoreMSE[, 1])^2)
print(paste("MSE of real test data (3 month prediction): ", mse_score))

[1] "tree number: 50" "depth: 3"        "shrinkage: 0.1" 


Using 50 trees...




[1] "MSE of the 70% train data (3 month prediction):  5.87502419829213e-06"


Using 50 trees...




[1] "MSE of the rest 30% data (3 month prediction):  0.000135230432558623"


Using 50 trees...




[1] "MSE of real test data (3 month prediction):  31.5507183491767"


In [34]:
# 12 month predition of gradient Boosting
train_x = train_x_12
train_y = train_y_12
test_x = test_x_12
test_y = test_y_12
score_y = score_y_12

train_data = train_data_12
test_data = test_data_12
data_scoreMSE = data_score_12

set.seed(12)

star = Sys.time()
train_control <- trainControl(method = "timeslice",
              initialWindow = floor(0.7*nrow(train_x)),
              horizon = 12,skip =3,
              fixedWindow = TRUE,
              allowParallel = TRUE)

#tune grid
gbmGrid <- expand.grid(
  interaction.depth = seq(1, 5),
  n.trees = c(50,100,200),
  shrinkage = c(0.01,0.1),
  n.minobsinnode = 10
)


model <- train(train_x, train_y,
   method = "gbm",
   tuneGrid=gbmGrid,
   trControl = train_control);
end = Sys.time()


print(paste("total cv time", end-star))
print(model$bestTune)



Iter   TrainDeviance   ValidDeviance   StepSize   Improve
     1        0.0008            -nan     0.1000    0.0001
     2        0.0007            -nan     0.1000    0.0001
     3        0.0007            -nan     0.1000    0.0000
     4        0.0006            -nan     0.1000    0.0000
     5        0.0006            -nan     0.1000    0.0000
     6        0.0006            -nan     0.1000    0.0000
     7        0.0005            -nan     0.1000    0.0000
     8        0.0005            -nan     0.1000    0.0000
     9        0.0005            -nan     0.1000    0.0000
    10        0.0005            -nan     0.1000    0.0000
    20        0.0003            -nan     0.1000    0.0000
    40        0.0002            -nan     0.1000   -0.0000
    50        0.0002            -nan     0.1000    0.0000

[1] "total cv time 7.26261973381042"
   n.trees interaction.depth shrinkage n.minobsinnode
22      50                 3       0.1             10


In [35]:
gbm_model <- gbm(train_y ~ ., data = train_data, distribution = "gaussian", n.trees = model$bestTune$n.trees, interaction.depth = model$bestTune$interaction.depth,shrinkage = model$bestTune$shrinkage)

gb_12 = gbm_model
print(paste(c('tree number:','depth:','shrinkage:'),c(model$bestTune$n.trees, model$bestTune$interaction.depth,model$bestTune$shrinkage)))
# train mse
predictions <- predict(gbm_model, newdata = train_data, type = "response")
mse <- mean((predictions - train_data[, 1])^2)
print(paste("MSE of the 70% train data (12 month prediction): ", mse))

#test mse
predictions <- predict(gbm_model, newdata = test_data, type = "response")
mse <- mean((predictions - test_data[, 1])^2)
print(paste("MSE of the rest 30% data (12 month prediction): ", mse))

#score mse
predictions <- predict(gbm_model, newdata = data_scoreMSE, type = "response")
mse_score <- mean((predictions - data_scoreMSE[, 1])^2)
print(paste("MSE of real test data (12 month prediction): ", mse_score))

[1] "tree number: 50" "depth: 3"        "shrinkage: 0.1" 


Using 50 trees...




[1] "MSE of the 70% train data (12 month prediction):  6.38386684303676e-06"


Using 50 trees...




[1] "MSE of the rest 30% data (12 month prediction):  0.00013954933213812"


Using 50 trees...




[1] "MSE of real test data (12 month prediction):  31.5607297502261"


## Conclusion

compare with all of different models, GB model performed the best than anyther model in all 3 period predition, significantly decreasing the test error.

as for lasso, ridge and rf model, t